Hybrid Search RAG Pipeline in LlamaIndex

This notebook demonstrates how to build a Hybrid Search Retrieval Augmented Generation (RAG) pipeline using Open Source Models using HuggingFace and FastEmbeddings with llama-index

Setup

First, install the necessary packages:

Install Necessary Packages and save Access Tokens:

In [1]:
!pip install llama-index-vector-stores-chroma
!pip install llama-index
!pip install llama-index-embeddings-fastembed

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached mmh3-4.1.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
     ---------------------------------------- 0.0/51.7 kB ? eta -:--:--
     ---------------------------------------- 5

  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: Could not build wheels for chroma-hnswlib, which is required to install pyproject.toml-based projects


  Using cached llama_index_core-0.10.67-py3-none-any.whl.metadata (2.4 kB)
  Using cached openai-1.41.0-py3-none-any.whl.metadata (22 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached dirtyjson-1.0.8-py3-none-any.whl.metadata (11 kB)
  Using cached tiktoken-0.7.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached jiter-0.5.0-cp312-none-win_amd64.whl.metadata (3.7 kB)
  Using cached marshmallow-3.21.3-py3-none-any.whl.metadata (7.1 kB)
Using cached llama_index_core-0.10.67-py3-none-any.whl (15.5 MB)
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB 8.6 MB/s eta 0:00:01
   -------------------------- ------------- 0.8/1.2 MB 10.2 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 9.5 MB/s eta 0:00:00
Using 

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      C:\Apps\Anaconda\Lib\site-packages\setuptools\__init__.py:81: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
      running bdist_wheel
      running build
      running build_ext
      cythoning src/Stemmer.pyx to src\Stemmer.c
      c:\users\rijul bhardwaj\appdata\local\temp\pip-install-znsu__2s\pystemmer_c0180e9a40b04fbb86deac435969a95a\.eggs\cython-0.29.37-py3.12.egg\Cython\Compiler\Main.py:3

In [2]:

!pip install llama-index-llms-huggingface-api

   ---------------------------------------- 0.0/402.8 kB ? eta -:--:--
   --- ------------------------------------ 30.7/402.8 kB 1.3 MB/s eta 0:00:01
   ------- -------------------------------- 71.7/402.8 kB 1.3 MB/s eta 0:00:01
   -------------------- ------------------- 204.8/402.8 kB 2.1 MB/s eta 0:00:01
   ------------------------------------ --- 368.6/402.8 kB 2.5 MB/s eta 0:00:01
   ---------------------------------------- 402.8/402.8 kB 2.8 MB/s eta 0:00:00


Set Up Hugging Face API Token
We'll need a Hugging Face API token to access pre-trained models from their hub.

In [2]:
import os
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass("API:")

# Set the API token in the environment variable
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

API:··········

Load the Medical Documents

In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()

TypeError: ForwardRef._evaluate() missing 1 required keyword-only argument: 'recursive_guard'

Set Up FastEmbeddings Embeddings and HuggingFace LLM
Here, we define the models used for text embedding (FastEmbeddings) and retrieval (Hugging Face LLM).



In [ ]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

llm = HuggingFaceInferenceAPI(
    model_name="HuggingFaceH4/zephyr-7b-alpha", token=HUGGINGFACEHUB_API_TOKEN
)
     

Define LLM and Embedding in Settings
By default LlamaIndex uses OpenAI, so we need to override the settings

In [ ]:
from llama_index.core import Settings

Settings.llm = llm

Settings.embed_model = embed_model

Index your document
First we save the data in disk

Create a Persist directory where the data will be stored
Define a unique collection for each index.
Store the data in StorageContext

In [ ]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

Load the index
Here, we load the pre-built document index. Notice, when you load, we don't use documents

In [ ]:

db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("how to prevent heart diseases")

In [ ]:
response.response

Hybrid Search
This section demonstrates how to perform a hybrid search combining text retrieval and vector similarity search.

In [ ]:

query_engine = index.as_query_engine(
    similarity_top_k=2, sparse_top_k=12, vector_store_query_mode="hybrid"
)


In [ ]:
from IPython.display import display, Markdown

response = query_engine.query(
    "what causes heart attacks"
)

display(Markdown(str(response)))
     